In [1]:
import json
import requests
from github import Github
import mysql.connector
from mysql.connector import errorcode
import sys
from os.path import expanduser
repos=[]
home=expanduser("~")
#Access token for the Github API
f=open(home+'/coursework/github token')
access_token=f.readlines()[0].rstrip()

#Helper function for getting responses

def getResponse(url):
    #url=url+"?access_token="+access_token+"&per_page=100"
    response=requests.get(url)
    d=response.headers.items()
    #print 'response.status_code is ',response.status_code
    link=""
    for (k,v) in response.headers.items():
        #print k,' : ',v
        if k.lower()=="link":
            link=v
    return response.status_code,link,response.json()


#This API returns the basic properties of a user 
def getUserProperties(userid):
    link="https://api.github.com/users/"+userid+"?access_token="+access_token
    response=requests.get(link)
    #print 'response is ',response.json()
    data=response.json()
    print 'getting properties for ',userid
    return (data["public_repos"],data["public_gists"],data["followers"],data["following"])

#This API would return the the link in the 
#LinkHeader and a boolean value of True if that link was not the last

def followLink(linkHeader):
    links=linkHeader.split(';')
    link1=links[0]
    temp=links[1].split(',')
    link2=temp[1]
    link2=link2.lstrip()
    link2=link2.rstrip()
    link2=link2[1:-1]
    link1=link1[1:-1]
    #print 'in followLinks link1 is : ',link1 ,' link2 is : ',link2,'\n\n'
    if link1==link2:
        return (link1,False)
    else:
        return (link1,True)
    

#API to get the number of followers of a user, checked works
def getNumberOfProperties(link):
    flag=True
    #link=link+'?access_token='+access_token+'&per_page=300'
    num=0
    while(1):
       
        linkHeader=""
        response=requests.get(link)
        #print 'link is ',link
        num+=len(response.json())
        if flag==False:
            break
        for (k,v) in response.headers.items():
            if k.lower()=='link':
                linkHeader=v
        if(linkHeader==""):
            #means that there is only one page of followers
            return len(response.json())
        (link,flag)=followLink(linkHeader)
    #print 'in numprop returning ',num
    return num   
        

def getOtherProperties(userid):
    l1="https://api.github.com/users/"+userid+'/'
    l2='?access_token='+access_token+'&per_page=300'
    #print 'trying 1'
    starred=getNumberOfProperties(l1+'starred'+l2)
    #print 'starred is ',starred
    subscriptions=getNumberOfProperties(l1+'subscriptions'+l2)
    #print 'subscriptions is ',subscriptions
    eventsDoneByTheUser=getNumberOfProperties(l1+'events'+l2)
    receivedEvents=getNumberOfProperties(l1+'received_events'+l2)
    organizations=getNumberOfProperties(l1+'orgs'+l2)
    
    #print 'sucessfully found all properties'
    return (starred,subscriptions,eventsDoneByTheUser,receivedEvents,organizations)

def printInfo(user,body):
    for (k,v) in user.items():
        print k,' : ',v
    print 'body : ',body
    print '\n\n'
        


f = open('repos.txt', 'r')
for line in f:
    line=line.split('\n')[0]
    line=line+'?access_token='+access_token
    print 'line = ',line
    repos.append(line)


    
#Connecting to the database
config = {
  'user': 'root',
  'password': 'Resnick1',
  'host': '127.0.0.1',
  'database': 'UserProfiles',
  'raise_on_warnings': True,
}

#Different Queries for MySQL
#Add a comment for a new userid
#add_comment = ("INSERT INTO comments "
           #"(login,id,commented_repo,avatar_url,gravatar_id,followers_url,following_url,gists_url,starred_url,subscriptions_url,organizations_url,repos_url,events_url,received_events_url,type,body) "
           #"VALUES (%s, %s, %s, %s, %s,%s, %s, %s, %s, %s,%s, %s, %s, %s, %s,%s)")
add_comment = ("INSERT INTO pullRequestComments "
           "(login,id,commented_repo,followers_url,following_url,num_followers,num_following,num_gists,num_starred,num_subscriptions,num_organizations,num_repos,num_events_done, num_received_events,body) "
           "VALUES (%s, %s, %s,%s,%s, %s, %s,%s,%s, %s, %s,%s,%s, %s, %s)")
#Update a comment for a existing user with the separator #NEWCOMMENT#
update_comment=("UPDATE pullRequestComments SET body=CONCAT(body,%s) where id = %s")
#Query to check if a user with a id already exists or not
checkId = ("SELECT COUNT(*) FROM pullRequestComments WHERE id=%s")

#add_pullreq=("INSERT INTO pullRequests "
#           "(login,userid,pull_request_id,commented_repo,body,pull_request_Status,pull_request_created_date,pull_request_closed_date,pull_request_merged_date) "
#           "VALUES (%s, %s, %s,%s,%s, %s, %s,%s,%s, %s, %s,%s%s, %s, %s)")
#Connect to the MySql server.

try:
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
    cnx.close()
    sys.exit()
    
#Iterate through the repositories

for repo in repos:
    #print 'repo is ',repo,'hi'
    repo=repo+"&per_page=300"
    (code,link,responses)=getResponse(repo)
    flag=True
    print 'Got Status ',code,' for ',repo
    if code!=200:
        continue
    print 'Processing ',repo
    while(flag):    
        
        #Now we iterate through the current responses
        
        for response in responses:
            try:
                user=response["user"]
                body=response["body"]
                userid=user["id"]
                userLogin=user["login"]
                cursor.execute(checkId,(userid,))
                (userExists,)=cursor.fetchone()
                #printInfo(user,body)(login,id,commented_repo,followers_url,following_url,num_followers,num_following,num_gists,num_starred,num_subscriptions,num_organizations,num_repos,num_events_done, num_received_events_url,body)
                try:
                    print 'trying userExists ',userExists,' userLogin is ',userLogin
                    if userExists==0:
                        #First we get the user properties
                        (num_repos,num_gists,num_followers,num_following)=getUserProperties(userLogin)
                        #print 'calling other properties'
                        (num_starred,num_subscriptions,num_events_done, num_received_events,num_organizations)=getOtherProperties(userLogin)
                        cursor.execute(add_comment,(userLogin,userid,repo,user["followers_url"],user["following_url"],num_followers,num_following,num_gists,num_starred,num_subscriptions,num_organizations,num_repos,num_events_done, num_received_events,body))
                        print 'inserted new '
                    else:
                        print 'trying to update'
                        new_comment=" #NEWCOMMENT# "+body
                        cursor.execute(update_comment,(new_comment,userid))
                        print 'updated'    
                    cnx.commit()
                except:
                    print 'failed to insert/update for link : ',link
                    pass
            except:
                print 'failed to insert/update for link : ',link
                pass
        #link1=link1+'?access_token='+access_token
        flag=False
        if link!="":
            print 'going forward'
            (link1,flag)=followLink(link)
            (code,link,responses)=getResponse(link1)
            #print 'just hit ',link1,' with code ',code,'\n\n'
            if(code!=200):
                print 'Code : ',code,'Start from here next time Cannot continue for link ',link1
                break;
      
cursor.close()
cnx.close()
print 'done now'

line =  https://api.github.com/repos/nodejs/node/pulls/comments?access_token=6fca6b10547db86c496e728cde80b715f7e4099a
line =  https://api.github.com/repos/mrdoob/three.js/pulls/comments?access_token=6fca6b10547db86c496e728cde80b715f7e4099a
Got Status  500  for  https://api.github.com/repos/nodejs/node/pulls/comments?access_token=6fca6b10547db86c496e728cde80b715f7e4099a&per_page=300
Got Status  200  for  https://api.github.com/repos/mrdoob/three.js/pulls/comments?access_token=6fca6b10547db86c496e728cde80b715f7e4099a&per_page=300
Processing  https://api.github.com/repos/mrdoob/three.js/pulls/comments?access_token=6fca6b10547db86c496e728cde80b715f7e4099a&per_page=300
trying userExists  0  userLogin is  zz85
getting properties for  zz85
inserted new 
trying userExists  0  userLogin is  broady
getting properties for  broady
inserted new 
trying userExists  1  userLogin is  broady
trying to update
updated
trying userExists  1  userLogin is  broady
trying to update
updated
trying userExists  